In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('matches.csv')


In [3]:
new_df = df[['team1', 'team2', 'winner', 'toss_decision', 'toss_winner']]


In [4]:
new_df.dropna(inplace=True)


/Users/anannyachuli/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [5]:
X = new_df[['team1', 'team2', 'toss_decision', 'toss_winner']]
y = new_df[['winner']]

In [6]:
all_teams = {}

In [7]:
cnt=0
for i in range(len(df)):
    if df.loc[i]['team1'] not in all_teams:
        all_teams[df.loc[i]['team1']] = cnt
        cnt += 1
        
    if df.loc[i]['team2'] not in all_teams:
        all_teams[df.loc[i]['team2']] = cnt
        cnt += 1


In [10]:
all_teams = np.array(list(all_teams.values()))


In [11]:
if all_teams.ndim > 1:
    all_teams = all_teams.flatten()

if len(all_teams) == 0:
    raise ValueError("all_teams cannot be empty")

le = LabelEncoder()
le.fit(all_teams)

LabelEncoder()

In [12]:
from sklearn.preprocessing import LabelEncoder
teams = LabelEncoder()
teams.fit(all_teams)

LabelEncoder()

In [13]:
encoded_teams = teams.transform(all_teams)

In [17]:
import pickle as pkl


In [19]:
X = np.array(X)
y = np.array(y)

In [21]:
new_labels = list(set(X[:, 0]) | set(X[:, 1]) | set(X[:, 3])) # find new labels
teams.fit_transform(new_labels) # update the encoder to include new labels

array([ 8,  7,  9,  2, 10, 11, 12,  3,  1,  5,  6,  0,  4, 13])

In [22]:
X[:, 0] = teams.transform(X[:, 0])
X[:, 1] = teams.transform(X[:, 1])
X[:, 3] = teams.transform(X[:, 3])


In [23]:
y[:, 0] = teams.transform(y[:, 0])

In [24]:

fb = {'field' : 0, 'bat' : 1}
for i in range(len(X)):
    X[i][2] = fb[X[i][2]] 

In [25]:

X = np.array(X, dtype='int32')
y = np.array(y, dtype='int32')
y_backup = y.copy()

y = y_backup.copy()


In [27]:

ones, zeros = 0,0
for i in range(len(X)):
    if y[i] == X[i][0] :
        if zeros <= 375:
            y[i] = 0
            zeros += 1
        else:
            y[i] = 1
            ones += 1
            t = X[i][0]
            X[i][0] = X[i][1] 
            X[i][1] = t
        
    
        
    if y[i] == X[i][1] :
        if ones <= 375:
            y[i] = 1
            ones += 1
        else:
            y[i] = 0
            zeros += 1
            t = X[i][0]
            X[i][0] = X[i][1] 
            X[i][1] = t

In [28]:

print(np.unique(y, return_counts=True))

(array([0, 1], dtype=int32), array([264, 369]))


In [29]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.05)



In [30]:

from sklearn.svm import SVC
model1 = SVC().fit(X_train, y_train)
model1.score(X_test, y_test)

/Users/anannyachuli/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.5625

In [31]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier().fit(X_train, y_train)
model2.score(X_test, y_test)

0.59375

In [32]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=250).fit(X, y)
model3.score(X_test, y_test)

/var/folders/8m/3v5jbdlx00n662l3wk4497x40000gn/T/ipykernel_17592/1874727435.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3 = RandomForestClassifier(n_estimators=250).fit(X, y)


0.84375

In [33]:
test = np.array([2,4, 1, 4]).reshape(1,-1)
model1.predict(test)
model2.predict(test)
model3.predict(test)



array([1], dtype=int32)

In [1]:
import pickle as pkl

def predict_winner(team1, team2, toss_win, choose):
with open('model.pkl', 'wb') as f:
    pkl.dump(model3, f)
with open('model.pkl', 'rb') as f:
    model = pkl.load(f)
with open('vocab.pkl', 'wb') as f:
    pkl.dump(encoded_teams, f)
with open('inv_vocab.pkl', 'wb') as f:
    pkl.dump(all_teams, f)

In [37]:
model.predict(test)


array([1], dtype=int32)